In [1]:
# Set the working directory to the parent directory
import sys
sys.path.append('..')
sys.dont_write_bytecode = True

# Import relevant custom libraries
from src.eda import data_info
from src.evaluation import ValidationEvaluation

# Import relevant libraries
import pandas as pd
import warnings
from IPython.display import display
import matplotlib.pyplot as plt
import math
import os
import glob
import numpy as np

# Config
pd.set_option('display.max_columns', None) # Ensure all columns are displayed
warnings.filterwarnings("ignore")

# Read relevant files
X_train = pd.read_feather("../data/processed/X_train.feather")
X_train_validate = pd.read_feather("../data/processed/X_train_validate.feather")

# Get data info
var_info = data_info(X_train)
all_cols = X_train.columns
real_cols = var_info[var_info["var_type"]=="numerical"]["var_name"].tolist()
binary_cols = var_info[var_info["var_type"]=="binary"]["var_name"].tolist()

# Read relevant files
X_validate = pd.read_feather("../data/processed/X_validate.feather")
y_validate = pd.read_feather("../data/processed/y_validate.feather")

# Initialize the validation evaluation
valeval = ValidationEvaluation(X_validate, y_validate, real_cols, binary_cols, all_cols, dp_sgd=True)
    
# Read the log file
log_path = "../logs/dpsgd_tune_log.txt"

# Extract the latest successful Bayesian versions
latest_successful_versions = valeval.extract_latest_successful_bayesian_versions(log_path)
print(latest_successful_versions)

# Evaluate the model performance
eval_results = valeval.evaluate_model_performance(latest_successful_versions)

{'202504242353': ('AUC', 1.0, 1e-05), '202504250426': ('Precision', 1.0, 1e-05), '202504230006': ('F1-Score', 1.0, 1e-05), '202504250552': ('Recall', 1.0, 1e-05), '202504250018': ('AUC', 3.0, 1e-05), '202505042114': ('F1-Score', 3.0, 1e-05), '202504250447': ('Precision', 3.0, 1e-05), '202504252301': ('Recall', 3.0, 1e-05), '202504250026': ('AUC', 5.0, 1e-05), '202504230052': ('F1-Score', 5.0, 1e-05), '202504250457': ('Precision', 5.0, 1e-05), '202504250625': ('Recall', 5.0, 1e-05), '202504242314': ('AUC', 0.5, 1e-05), '202504222233': ('F1-Score', 0.5, 1e-05), '202504250252': ('Precision', 0.5, 1e-05), '202504250502': ('Recall', 0.5, 1e-05)}
Evaluating version 202504242353
Evaluating version 202504250426
Evaluating version 202504230006
Evaluating version 202504250552
Evaluating version 202504250018
Evaluating version 202505042114
Evaluating version 202504250447
Evaluating version 202504252301
Evaluating version 202504250026
Evaluating version 202504230052
Evaluating version 202504250457

In [2]:
selected_versions = eval_results[eval_results['epsilon']>=0.5].sort_values(by=["tuned_by", "epsilon"], ascending=True)[['epsilon', 'tuned_by']].reset_index()
display(selected_versions)

,version,epsilon,tuned_by
0,202504250252,0.5,Precision
1,202504250426,1.0,Precision
2,202504250447,3.0,Precision
3,202504250457,5.0,Precision
4,202504250502,0.5,Recall
5,202504250552,1.0,Recall
6,202504252301,3.0,Recall
7,202504250625,5.0,Recall
8,202504222233,0.5,F1-Score
9,202504230006,1.0,F1-Score


In [15]:
def get_convergence_status(version, dpsgd=True):
    # Read result file
    folder = "../experiments/tracking"

    # Read the existing files with the same version prefix
    if dpsgd:
        pattern = os.path.join(folder, f"{version}_noise*.csv")
        for file in glob.glob(pattern):
            results_df = pd.read_csv(file)
    else:
        results_df = pd.read_csv(os.path.join(folder, f"{version}_baseline.csv"))

    # Check convergence
    loss_history = np.array(results_df["val_loss"].tolist())
    n = len(loss_history)
    tail_len = 20 #int(0.1 * n)

    # Ensure tail_len is at least 2 to compute slope
    if n < 20:
        raise ValueError("Too few points in the tail to compute slope.")

    # Compute the slope of the last 10% of the loss history
    y = loss_history[-tail_len:]
    x = np.arange(tail_len)

    # Fit a line to the last 10% of the loss history
    slope = np.polyfit(x, y, 1)[0]  # degree 1 polynomial fit

    rel_change = abs(y[-1] - y[0]) / max(abs(y[0]), 1e-8)  # avoid divide-by-zero

    return slope, rel_change

In [4]:
slope_rel_change = selected_versions["version"].apply(get_convergence_status)
selected_versions["slope"] = slope_rel_change.apply(lambda x: x[0])
selected_versions["rel_change"] = slope_rel_change.apply(lambda x: x[1])

In [5]:
selected_versions.sort_values(["slope"], ascending=True)

,version,epsilon,tuned_by,slope,rel_change
0,202504250252,0.5,Precision,-0.081899,0.067766
4,202504250502,0.5,Recall,-0.020839,0.011756
12,202504242314,0.5,AUC,-0.004836,0.013047
5,202504250552,1.0,Recall,-0.002415,0.017752
2,202504250447,3.0,Precision,-0.001901,0.008809
3,202504250457,5.0,Precision,-0.001369,0.005984
7,202504250625,5.0,Recall,-0.000917,0.013935
9,202504230006,1.0,F1-Score,-0.000732,0.005925
6,202504252301,3.0,Recall,-0.000661,0.024295
15,202504250026,5.0,AUC,-0.000608,0.052258


In [6]:
eval_results.reset_index().sort_values("version", ascending=True)["version"]

11    202504222233
8     202504230006
10    202504230052
15    202504242314
12    202504242353
13    202504250018
14    202504250026
3     202504250252
0     202504250426
1     202504250447
2     202504250457
7     202504250502
4     202504250552
6     202504250625
5     202504252301
9     202505042114
Name: version, dtype: object

In [13]:
# Initialize the validation evaluation
valeval = ValidationEvaluation(X_validate, y_validate, real_cols, binary_cols, all_cols, dp_sgd=False)
    
# Read the log file
log_path = "../logs/baseline_tune_log.txt"

# Extract the latest successful Bayesian versions
latest_successful_versions = valeval.extract_latest_successful_bayesian_versions(log_path)
print(latest_successful_versions)

# Evaluate the model performance
eval_results = valeval.evaluate_model_performance(latest_successful_versions)

{'202504180245': 'AUC', '202504180314': 'Precision', '202504180259': 'F1-Score', '202504180327': 'Recall'}
Evaluating version 202504180245
Evaluating version 202504180314
Evaluating version 202504180259
Evaluating version 202504180327


In [16]:
selected_versions = eval_results.reset_index()
slope_rel_change = selected_versions["version"].apply(get_convergence_status, dpsgd=False)
selected_versions["slope"] = slope_rel_change.apply(lambda x: x[0])
selected_versions["rel_change"] = slope_rel_change.apply(lambda x: x[1])

In [17]:
selected_versions

,version,precision,recall,f1_score,auc,accuracy,hidden_dims,batch_size,dropout_rate,learning_rate,lam,gamma,threshold,tuned_by,slope,rel_change
0,202504180314,0.771379,0.191757,0.307158,0.699887,0.825958,[64],108.0,0.237561,0.070543,0.044082,0.084559,16.706953,Precision,-0.001706,0.003224
1,202504180327,0.432645,0.645119,0.517938,0.796319,0.758401,[64],123.0,0.378855,0.059096,0.000100,0.999000,0.038421,Recall,-0.000015,0.010256
2,202504180259,0.496141,0.585683,0.537207,0.809764,0.796980,[64],64.0,0.352388,0.020000,0.000100,0.999000,0.045073,F1-Score,-0.000066,0.024831
3,202504180245,0.499449,0.589588,0.540788,0.812854,0.798551,[64],128.0,0.000000,0.100000,0.000100,0.999000,0.039729,AUC,-0.000215,0.086768
